In [1]:
import torch
import requests
import os
from pathlib import Path
URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [5]:
%load_ext autoreload

%autoreload 2

In [15]:
from glasses.utils.PretrainedWeightsProvider import PretrainedWeightsProvider


provider = PretrainedWeightsProvider()

provider['resnet50']

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (block): Sequential(
          (0): ResNetBottleneckBlock(
            (block): Sequential(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act2): ReLU(inplace=True)
          

In [6]:
import io
from utils.ModuleTransfer import ModuleTransfer
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152
from nn.models.classification.resnet import ResNet
from tqdm.autonotebook import tqdm

zoo = {
    resnet18: 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    resnet34: 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    resnet50: 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    resnet101: 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    resnet152: 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

zoo_models_mapping = {
    
    resnet18: ResNet.resnet18,
    resnet34: ResNet.resnet34,
    resnet50: ResNet.resnet50,
    resnet101: ResNet.resnet101,
    resnet152: ResNet.resnet152,
}

for src_def, dst_def in zoo_models_mapping.items():
    url = zoo[src_def]
    print(f"Getting weights at url={url}")
    
    r = requests.get(url, stream=True)
    
    with open('../temp.pth', 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        bar = tqdm(r.iter_content(chunk_size=1024), total=total_length/1024)
        for chunk in bar: 
            if chunk:
                f.write(chunk)
                f.flush()

    src = src_def(False)
    src.load_state_dict(torch.load('../temp.pth'))
    dst = dst_def()

    x = torch.rand((1, 3, 224, 224))

    src.eval()
    dst.eval()
    
    a = src(x)
    b = dst(x)

    assert not torch.equal(a, b)
    
    
    ModuleTransfer(src, dst)(x)

    a = src(x)
    b = dst(x)

    assert torch.equal(a, b)

Getting weights at url=https://download.pytorch.org/models/resnet18-5c106cde.pth



Getting weights at url=https://download.pytorch.org/models/resnet34-333f7ec4.pth



Getting weights at url=https://download.pytorch.org/models/resnet50-19c8e357.pth



Getting weights at url=https://download.pytorch.org/models/resnet101-5d3b4d8f.pth



Getting weights at url=https://download.pytorch.org/models/resnet152-b121ed2d.pth


In [4]:
!ls

data	     interpretability  nn		 __pycache__
__init__.py  models.ipynb      playground.ipynb  utils
